### Importing
`multiprocessing` is in the Python Standard Library. It provides an easy-to-use interface for running multi-threaded jobs on a single compute node.

In [1]:
from multiprocessing import Pool
import numpy as np

### Defining worker function
`worker_pi` is the function to be run in parallel on each worker. It is the MonteCarlo approximation of $\pi$.

In [2]:
from random import random

def worker_pi(n):
    s = 0
    for _ in range(n):
        x, y = random(), random()
        s += x * x + y * y <= 1.0
    return 4 * s / n

### Setting problem size

Use the max number of workers as determined by the SLURM environment variable `SLURM_CPUS_PER_TASK`. Set the number of elements per worker.

In [3]:
from helpers import CPUS_PER_TASK
N = int(1e7)

print("{0} threads, {1} elements per thread".format(CPUS_PER_TASK,N))

8 threads, 10000000 elements per thread


### Everybody into the pool
`Pool(processes)` creates a pool object with `processes` number of processes. If omitted, defaults to the number of processes available.

`pool` has a `map` method, mapping the function `worker_pi` to each element of the iterable. The number of elements for the serial routine has to be multiplied by the number of the threads available for the same amount of work as the parallel routine.

In [4]:
pool = Pool(CPUS_PER_TASK)

In [5]:
%timeit result = np.mean(pool.map(worker_pi, [N for _ in range(CPUS_PER_TASK)]))

1 loops, best of 3: 7.24 s per loop


In [6]:
%timeit result = worker_pi(N * CPUS_PER_TASK)

1 loops, best of 3: 37.2 s per loop


In [7]:
print(worker_pi(N * CPUS_PER_TASK), np.mean(pool.map(worker_pi, [N for _ in range(CPUS_PER_TASK)])))

3.1414107 3.1415618


In [8]:
pool.close()
pool.join()